<a href="https://colab.research.google.com/github/dmsophia/ponderadas_mod11/blob/main/ponderada-sem4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação de bibliotecas e leitura dos dados

In [42]:
!pip install gdown

In [26]:
!pip install keras-tuner

In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import plotly.graph_objs as go
import plotly.express as px
import keras_tuner as kt

In [43]:
import gdown

file_id = "1P5A3kDo2C74z2h-qjKiJ-sE8jeEHyhM_"
url = f"https://drive.google.com/uc?id={file_id}"
output = "creditcard.csv"
gdown.download(url, output, quiet=False)
df = pd.read_csv(output)

df.head()

Downloading...
From (original): https://drive.google.com/uc?id=1P5A3kDo2C74z2h-qjKiJ-sE8jeEHyhM_
From (redirected): https://drive.google.com/uc?id=1P5A3kDo2C74z2h-qjKiJ-sE8jeEHyhM_&confirm=t&uuid=7863f40a-2217-4449-af4f-f083b093f033
To: /content/creditcard.csv
100%|██████████| 151M/151M [00:01<00:00, 129MB/s]


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


O conjunto de dados contém as seguintes colunas:

* Time: O tempo decorrido entre essa transação e a primeira transação no conjunto de dados.

* V1 a V28: Estes são os resultados de uma transformação PCA (Análise de Componentes Principais). Eles representam as características da transação, com dimensionalidade reduzida.

* Amount: O valor da transação.

* Class: O rótulo da transação, onde 0 indica uma transação não fraudulenta e 1 indica uma transação fraudulenta.

Nosso objetivo é construir um modelo de rede neural para detectar transações fraudulentas (Class = 1) e, em seguida, avaliar seu desempenho usando várias métricas.

# Pré - processamento

Vamos normalizar as colunas Amount e Time, pois redes neurais tendem a ter melhor desempenho com entradas normalizadas. Também vamos dividir o conjunto de dados em conjuntos de treinamento e teste.

In [30]:
X = df.drop(columns=['Class'])
y = df['Class']

scaler = StandardScaler()
X[['Amount', 'Time']] = scaler.fit_transform(X[['Amount', 'Time']])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

((199364, 30), (85443, 30), (199364,), (85443,))

# Treinamento do modelo

## Primeiro modelo

In [31]:
model_original = Sequential([
    Dense(32, input_dim=30, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_original.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history_original = model_original.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=1)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 35s 5ms/step - accuracy: 0.9944 - loss: 0.0319 - val_accuracy: 0.9993 - val_loss: 0.0037
Epoch 2/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.9995 - loss: 0.0028 - val_accuracy: 0.9993 - val_loss: 0.0038
Epoch 3/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.9995 - loss: 0.0024 - val_accuracy: 0.9993 - val_loss: 0.0036
Epoch 4/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 44s 6ms/step - accuracy: 0.9995 - loss: 0.0023 - val_accuracy: 0.9993 - val_loss: 0.0035
Epoch 5/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 26s 4ms/step - accuracy: 0.9995 - loss: 0.0019 - val_accuracy: 0.9993 - val_loss: 0.0034
Epoch 6/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 26s 4ms/step - accuracy: 0.9995 - loss: 0.0020 - val_accuracy: 0.9993 - val_loss: 0.0035
Epoch 7/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 26s 4ms/step - accuracy: 0.9995 - loss: 0.0019 - val_accuracy: 0.9993 - val_loss: 0.0034
Epoch 8/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 32s 5ms/step - accuracy: 0.9996 - loss: 0

* Camada de Entrada: possui 30 neurônios (input_dim=30), correspondendo ao número de características (features) do conjunto de dados. Cada neurônio nesta camada representa uma das entradas (características) do conjunto de dados.
* Primeira Camada Oculta: 32 neurônios oferece um bom equilíbrio entre capacidade de aprendizado e complexidade do modelo. Um número moderado de neurônios permite que a rede aprenda padrões significativos sem superajustar os dados. Com a função de ativação ReLU por sua eficiência em evitar o problema do "vanishing gradient" e por sua simplicidade computacional.
* Segunda Camada Oculta: reduz o número de neurônios para 16. Essa redução ajuda a rede a aprender representações mais compactas e evita o risco de sobreajuste (overfitting). A função de ativação ReLU é novamente utilizada aqui para manter a consistência e continuar aproveitando suas vantagens na introdução de não-linearidade.
* Camada de Saída: possui um único neurônio porque a tarefa é de classificação binária (fraude ou não fraude). Esse neurônio produz um valor entre 0 e 1, que representa a probabilidade de uma transação ser fraudulenta. A função de ativação aplicada é sigmoid, ideal para a camada de saída em problemas de classificação binária, pois converte o output em uma probabilidade. Valores próximos a 0 indicam baixa probabilidade de fraude, enquanto valores próximos a 1 indicam alta probabilidade.
* Função de Perda: `binary_crossentropy` é adequada para problemas de classificação binária. Ela mede a diferença entre as previsões do modelo e os valores reais (rótulos), penalizando previsões incorretas. Essa função é preferida porque fornece uma métrica eficaz para ajustar o modelo de forma que ele seja capaz de distinguir entre as duas classes (fraude e não fraude) com alta precisão.
*Otimizador: Adam é escolhido por sua eficiência e capacidade de adaptar a taxa de aprendizado durante o treinamento. Ele combina as vantagens dos otimizadores Adagrad e RMSProp, resultando em uma convergência mais rápida e estável.
* Número de Épocas: foi definido como 10, o que geralmente é suficiente para que o modelo converja e aprenda padrões úteis nos dados sem superajustar.

In [32]:
y_pred_original = (model_original.predict(X_test) > 0.5).astype("int32")
precision_original, recall_original, f1_score_original, _ = precision_recall_fscore_support(y_test, y_pred_original, average='binary')
auc_roc_original = roc_auc_score(y_test, y_pred_original)

2671/2671 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step


## Modelo otimizado

In [33]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=16, max_value=128, step=16), activation='relu', input_dim=30))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=3,
    directory='my_dir',
    project_name='fraud_detection_tuning'
)

tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Trial 3 Complete [00h 09m 52s]
val_accuracy: 0.9993446071942648

Best val_accuracy So Far: 0.9993875026702881
Total elapsed time: 00h 30m 43s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
80                |96                |units
adam              |adam              |optimizer

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



6231/6231 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.9723 - loss: 0.0699 - val_accuracy: 0.9993 - val_loss: 0.0035
Epoch 2/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.9995 - loss: 0.0024 - val_accuracy: 0.9993 - val_loss: 0.0034
Epoch 3/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.9994 - loss: 0.0023 - val_accuracy: 0.9993 - val_loss: 0.0034
Epoch 4/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.9994 - loss: 0.0023 - val_accuracy: 0.9993 - val_loss: 0.0032
Epoch 5/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.9995 - loss: 0.0020 - val_accuracy: 0.9993 - val_loss: 0.0033
Epoch 6/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.9995 - loss: 0.0020 - val_accuracy: 0.9994 - val_loss: 0.0033
Epoch 7/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - accuracy: 0.9995 - loss: 0.0019 - val_accuracy: 0.9993 - val_loss: 0.0032
Epoch 8/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.9995 - loss: 0.0020 - val

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



6231/6231 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.9930 - loss: 0.0272 - val_accuracy: 0.9993 - val_loss: 0.0037
Epoch 2/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.9994 - loss: 0.0031 - val_accuracy: 0.9993 - val_loss: 0.0037
Epoch 3/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.9994 - loss: 0.0023 - val_accuracy: 0.9992 - val_loss: 0.0037
Epoch 4/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.9994 - loss: 0.0024 - val_accuracy: 0.9993 - val_loss: 0.0034
Epoch 5/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - accuracy: 0.9994 - loss: 0.0026 - val_accuracy: 0.9993 - val_loss: 0.0034
Epoch 6/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.9995 - loss: 0.0018 - val_accuracy: 0.9992 - val_loss: 0.0033
Epoch 7/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.9995 - loss: 0.0018 - val_accuracy: 0.9992 - val_loss: 0.0037
Epoch 8/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9995 - loss: 0.0018 - val

KeyboardInterrupt: 

In [35]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

model_optimized = tuner.hypermodel.build(best_hps)
history_optimized = model_optimized.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=1)

y_pred_optimized = (model_optimized.predict(X_test) > 0.5).astype("int32")
precision_optimized, recall_optimized, f1_score_optimized, _ = precision_recall_fscore_support(y_test, y_pred_optimized, average='binary')
auc_roc_optimized = roc_auc_score(y_test, y_pred_optimized)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 42s 6ms/step - accuracy: 0.9908 - loss: 0.0293 - val_accuracy: 0.9993 - val_loss: 0.0034
Epoch 2/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - accuracy: 0.9994 - loss: 0.0028 - val_accuracy: 0.9993 - val_loss: 0.0036
Epoch 3/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.9995 - loss: 0.0023 - val_accuracy: 0.9993 - val_loss: 0.0034
Epoch 4/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.9995 - loss: 0.0021 - val_accuracy: 0.9992 - val_loss: 0.0034
Epoch 5/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.9995 - loss: 0.0019 - val_accuracy: 0.9993 - val_loss: 0.0035
Epoch 6/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.9995 - loss: 0.0023 - val_accuracy: 0.9993 - val_loss: 0.0033
Epoch 7/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.9995 - loss: 0.0018 - val_accuracy: 0.9994 - val_loss: 0.0033
Epoch 8/10
6231/6231 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.9996 - loss: 0

* Camada de Entrada: começa com uma camada de entrada com 30 neurônios, correspondendo ao número de características (features) do conjunto de dados de entrada.
* Primeira Camada Oculta: é determinado pelo hiperparâmetro `units`, que é ajustado pelo `Keras Tuner` dentro de um intervalo entre 16 e 128, com passos de 16. Isso permite que o tuner experimente diferentes tamanhos de camada para encontrar a quantidade ideal de neurônios que equilibre a capacidade de aprendizado com a complexidade do modelo. A função de ativação ReLU é novamente escolhida devido à sua eficiência em evitar problemas como o "vanishing gradient" e sua habilidade em introduzir não-linearidade no modelo, o que é essencial para capturar padrões complexos nos dados.
* Segunda Camada Oculta: esta camada é fixa com 16 neurônios, o que representa uma escolha conservadora para evitar o sobreajuste enquanto ainda permite a rede aprender representações significativas dos dados.  Como na camada anterior, a função ReLU é usada para manter a consistência e aproveitar seus benefícios na introdução de não-linearidade.
* Camada de Saída: contém um único neurônio, adequado para a tarefa de classificação binária (fraude vs. não fraude). O neurônio de saída gera um valor entre 0 e 1, representando a probabilidade de uma transação ser fraudulenta. A função de ativação sigmoid é apropriada para a saída binária, pois converte o valor de saída em uma probabilidade.
* Função de Perda:  `binary_crossentropy` é a escolha padrão para problemas de classificação binária, como a detecção de fraudes. Ela mede a divergência entre as previsões do modelo e os rótulos verdadeiros, penalizando previsões incorretas para orientar o modelo a fazer previsões mais precisas.
* Otimizador: r é escolhido com base no ajuste de hiperparâmetros, onde o `Keras Tuner` testa tanto o `adam` quanto o `rmsprop` para determinar qual oferece o melhor desempenho. O `adam` é conhecido por sua eficiência e adaptabilidade, enquanto o `rmsprop` é eficaz em situações onde a taxa de aprendizado precisa ser ajustada dinamicamente durante o treinamento. A escolha entre os dois depende de qual otimizador leva a uma melhor convergência e desempenho no conjunto de validação.

**Ajuste Fino dos Hiperparâmetros: `Random Search`**

O ajuste fino dos hiperparâmetros foi feito usando a técnica de `Random Search` com o `Keras Tuner`. Essa técnica permite explorar de forma eficaz o espaço dos hiperparâmetros testando uma amostra aleatória de combinações dentro dos limites definidos. Embora o `Grid Search` também seja uma opção, o `Random Search` tende a ser mais eficiente em termos de tempo, especialmente quando há muitas combinações possíveis de hiperparâmetros.

In [36]:
df_metrics = pd.DataFrame({
    'Métrica': ['Precisão', 'Recall', 'F1-Score', 'AUC-ROC'],
    'Original': [precision_original, recall_original, f1_score_original, auc_roc_original],
    'Otimizado': [precision_optimized, recall_optimized, f1_score_optimized, auc_roc_optimized]
})

In [39]:
fig = px.bar(df_metrics.melt(id_vars='Métrica'),
             x='Métrica',
             y='value',
             color='variable',
             barmode='group',
             labels={'variable': 'Modelo', 'value': 'Valor'},
             title='Comparação das Métricas entre Modelos Original e Otimizado')

fig.update_layout(
    xaxis_title="Métricas",
    yaxis_title="Valor",
    legend_title="Modelo",
    title_x=0.5
)

fig.show()


O modelo original apresentou um desempenho ligeiramente superior ao modelo otimizado em todas as métricas. Isso pode indicar que, para este conjunto de dados específico e configuração inicial, o modelo original já estava bem ajustado, e as tentativas de otimização dos hiperparâmetros não resultaram em melhorias significativas, e até levaram a uma pequena degradação do desempenho.

Isso destaca a importância de avaliar cuidadosamente as métricas após a otimização e não assumir que um modelo otimizado será sempre melhor. Dependendo do contexto e dos objetivos específicos (por exemplo, priorizar precisão sobre recall), o modelo original pode ser a escolha mais adequada.